# Frank-Wolfe Algorithm

By ZincCat

$\min _{\mathbf{x} \in \mathbb{R}^{N}}\|\mathbf{y}-\mathbf{D} \mathbf{x}\|_{2}^{2}$

s.t. $\|\mathbf{x}\|_{?} \leq 1$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
p = 200
n = 300
np.random.seed(19890817)
D = np.random.normal(10, 5, (p, n))
y = np.random.normal(10, 5, p)
x0 = np.random.rand(n)

In [ ]:
def f(x):
    return np.linalg.norm(y-D@x)**2
def grad(x):
    return 2*D.T@(D@x-y)

In [ ]:
# 调用scipy求值
from scipy.optimize import minimize
cons1 = ({'type': 'ineq', 'fun': lambda x: 1 - np.linalg.norm(x, 1)})
res = minimize(f, x0, constraints=cons1, tol=1e-4, options={'maxiter': 1e4, 'disp': True})
minValue = res.fun
print("Scipy result:", res.fun)

In [ ]:
# 调用scipy求值
from scipy.optimize import minimize
cons2 = ({'type': 'ineq', 'fun': lambda x: 1 - np.linalg.norm(x, np.inf)})
res = minimize(f, x0, constraints=cons2, tol=1e-10, options={'maxiter': 1e4, 'disp': True})
minValue2 = res.fun
print("Scipy result:", res.fun)

In [ ]:
def fw(f, x, grad, maxIter, mode, log):
    xn = x.copy()
    for i in range(maxIter):
        value = f(xn)
        print(i, "th iteration, f(x)=", value)
        log.append(value)
        gamma = 2/(i+2)
        g = grad(xn)
        if mode == 1:
            d = np.argmax(np.abs(g))
            xn = (1-gamma)*xn
            xn[d] -= gamma * np.sign(g[d])
        elif mode == 'inf':
            d = -np.sign(g)
            xn += gamma * (d-xn)
    return xn

$l_\infty$ constraint

In [ ]:
maxIter = 3000000
linf = []
x = fw(f, x0/2/np.linalg.norm(x0, np.inf), grad, maxIter, 'inf', linf)

In [ ]:
plt.plot(np.log(linf))
plt.xlabel('Iterations')
plt.ylabel('$\ln (f(x_k)-f^*)$')
plt.savefig('wolfeInf')
plt.show()

$l_1$ constraint

In [ ]:
maxIter = 300000
l1 = []
x2 = fw(f, x0/2/np.linalg.norm(x0, 1), grad, maxIter, 1, l1)

In [ ]:
plt.plot(np.log(l1-minValue))
plt.xlabel('Iterations')
plt.ylabel('$\ln (f(x_k)-f^*)$')
plt.savefig('wolfe1')
plt.show()

projected gradient descent

In [ ]:
def P1(x):
    norm = np.linalg.norm(x, 1)
    if norm > 1:
        return x/norm
    return x
def Pinf(x):
    t = np.minimum(x, np.ones(n))
    return np.maximum(t, -np.ones(n))

In [ ]:
def linesearch_Armijo(f, x, g, d, alpha=0.4, beta=0.8):
    # backtrack linesearch using Armijo rules
    t = 10.0
    value = f(x)
    while f(x + t*d) > value + alpha*t*np.dot(g, d):
        t *= beta
    return t
def projectedDescent(f, x, grad, proj):
    # 投影梯度下降函数
    # 输入函数f, 目前x取值, 梯度函数, 要投影到的矩阵A
    # 输出下降后x取值, 步长t
    xn = x.copy()
    g = grad(xn)
    grad_norm = np.linalg.norm(g, 2)
    d = -g/grad_norm
    t = linesearch_Armijo(f, xn, g, d)
    xn += t*d
    return proj(xn), t

In [ ]:
# 绘图
time1 = []  # 记录时间步, 用于绘图
values1 = []  # 记录某一时间步下函数值, 用于绘图
Plot = True  # 是否绘图, 请保证此时alpha, beta均为单一取值
timestep = 0

x = x0.copy() #满足约束的初值

# 用于判定终止
count = 0 
eps = 1e-13
oldvalue = f(x)
maxIter = 200000  # 最大迭代次数

while True:
    value = f(x)
    print("Iteration:", timestep, "Value", value)
    # 用函数值改变量作为终止条件
    if abs(value - oldvalue) < eps:
        count += 1
    else:
        count = 0
    oldvalue = value
    if Plot:
        time1.append(timestep)
        values1.append(value)
    if timestep > maxIter or count >= 5:
        break
    x, t = projectedDescent(f, x, grad, Pinf)  # 此时使用无穷范数
    timestep += 1